In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
DATA_START = pd.to_datetime('2015-01-01')
DATASET_END = pd.to_datetime('2018-01-01')
NUM_SAMPLES = int(1e6)
MIN_APT_SIZE = 600

In [3]:
listing_dates = DATA_START + pd.to_timedelta(np.random.randint(0, 365*3, size=NUM_SAMPLES), unit='D')

In [4]:
house_qualities = np.random.exponential(size=NUM_SAMPLES)
years_since_remodel = np.random.exponential(scale=1e1,size=NUM_SAMPLES)
size_bonus = np.random.rand(NUM_SAMPLES) * 100
listing_price = np.clip(house_qualities * 400000 - years_since_remodel * 1000, 1000, None)

In [5]:
sqft = house_qualities * 1000 + size_bonus + MIN_APT_SIZE

In [6]:
bedroom_bonus = np.random.rand(NUM_SAMPLES) * 1000
bedrooms = np.floor(np.power((sqft + bedroom_bonus) * house_qualities // 300, .4))
#If more bedrooms per same space, fewer bathrooms can fit (hypothetical to make data more interesting)
bathroom_bonus = np.random.rand(NUM_SAMPLES) * 1000 * house_qualities - 1000 * bedrooms
bathrooms = np.floor(np.power(np.clip((sqft + bathroom_bonus) // 10, 0, None) + 1, .3))

In [7]:
#inspection

In [8]:
print('bedroom listing_price correlation,', np.corrcoef(bedrooms, listing_price)[0,1])

bedroom listing_price correlation, 0.9600297810667326


In [9]:
print('bathrooms listing_price correlation,', np.corrcoef(bathrooms, listing_price)[0,1])

bathrooms listing_price correlation, 0.3115627269940307


In [10]:
print('bedroom bathroom correlation,', np.corrcoef(bedrooms, bathrooms)[0,1])
print('bedroom min mean max', bedrooms.min(), bedrooms.mean(), bedrooms.max())
print('bathroom miin mean max',bathrooms.min(), bathrooms.mean(), bathrooms.max())

bedroom bathroom correlation, 0.1289277744494703
bedroom min mean max 0.0 1.516093 13.0
bathroom miin mean max 1.0 2.678189 8.0


In [11]:
(listing_dates - DATA_START).days.values.astype(float)

array([ 903.,  301.,  910., ...,  489.,  643., 1002.])

In [12]:
time_bonus = (listing_dates - DATA_START).days.values.astype(float)

In [13]:
bath_bed_penalty = bathrooms - bedrooms
bath_bed_ind = bathrooms > bedrooms

In [14]:
broker = np.random.randint(0, 2, NUM_SAMPLES)

In [15]:
sell_inv = bedrooms * bathrooms/bedrooms.std()/bathrooms.std() + bedrooms/bedrooms.std()
sell_inv += broker
sell_inv += time_bonus/time_bonus.std()
sell_inv += bathrooms/bathrooms.std() + sqft/sqft.std()
sell_inv += house_qualities*2
sell_inv += size_bonus/size_bonus.std()
sell_inv -= years_since_remodel/years_since_remodel.std()
sell_inv += np.random.rand(NUM_SAMPLES) + 10
sell_inv[bath_bed_ind] = sell_inv[bath_bed_ind] - bath_bed_penalty[bath_bed_ind]/2
sales_duration = np.floor(1/sell_inv * 3000).astype(int)

In [16]:
sales_duration.min(), sales_duration.mean(), sales_duration.std(), sales_duration.max()

(20, 139.562799, 40.36812847142655, 922)

In [17]:
sales_duration

array([ 76, 154, 134, ..., 192, 129, 116])

In [18]:
def plt_clipped(a):
    clip_max = np.percentile(a, 99)
    a = np.clip(a, a.min(), clip_max)
    #a = a[a < clip_max]
    plt.hist(a, bins=100)

In [19]:
sales_date = listing_dates + pd.to_timedelta(sales_duration,unit='D')

In [20]:
final_df = pd.DataFrame({'SalesDate':sales_date, 'ListingDate':listing_dates,
              'bedrooms':bedrooms,'bathrooms':bathrooms,'sqft':sqft,'years_since_remodel':years_since_remodel,
                        'broker': broker})

In [21]:
final_df.loc[final_df.SalesDate > DATASET_END, 'SalesDate'] = np.nan

In [22]:
cnt = 0
for ridx in np.random.choice(final_df.shape[0], final_df.shape[0]//6, replace = False):
    cnt += 1
    if cnt % 10000 == 0:
        print(cnt, cnt/final_df.shape[0]*6)
    if final_df.loc[ridx,'broker']:
        if np.random.rand() > .5:
            final_df.loc[ridx, 'sqft'] += 100
            final_df.loc[ridx, 'bedrooms'] = np.nan
        else:
            final_df.loc[ridx, 'bedrooms'] += 1
            final_df.loc[ridx, 'sqft'] = np.nan
    else:
        feature = np.random.randint(2, final_df.shape[1])
        final_df.iloc[ridx, feature] = np.nan
    final_df.loc[ridx,'broker'] = np.nan


10000 0.05
20000 0.1
30000 0.15
40000 0.2
50000 0.25
60000 0.3
70000 0.35000000000000003
80000 0.4
90000 0.44999999999999996
100000 0.5
110000 0.55
120000 0.6
130000 0.65
140000 0.7000000000000001
150000 0.75
160000 0.8


In [23]:
final_df.to_csv('houselistings_simulated.csv')

In [24]:
final_df.isnull().sum()

ListingDate                 0
SalesDate              117995
bathrooms               16627
bedrooms                58868
broker                 166666
sqft                    58027
years_since_remodel     16600
dtype: int64

In [25]:
final_df

,ListingDate,SalesDate,bathrooms,bedrooms,broker,sqft,years_since_remodel
0,2017-06-22,2017-09-06,4.0,3.0,0.0,3429.597547,18.955974
1,2015-10-29,2016-03-31,3.0,1.0,1.0,1156.326659,3.407768
2,2017-06-29,2017-11-10,3.0,1.0,1.0,1237.643572,1.684121
3,2016-01-11,2016-07-11,1.0,1.0,0.0,906.362675,13.295783
4,2015-08-31,2016-04-04,3.0,0.0,1.0,830.092667,22.250056
5,2015-02-25,2015-05-06,4.0,4.0,0.0,3834.721655,1.638502
6,2015-12-10,2016-03-12,3.0,3.0,0.0,2973.543188,2.274498
7,2017-09-21,NaT,2.0,2.0,0.0,1741.543443,5.489107
8,2017-02-14,2017-07-01,3.0,1.0,NaN,1657.706389,17.784693
9,2015-03-20,2015-11-25,3.0,0.0,0.0,751.573914,21.218993
